<h1>Training Custom Classifiers with IBM Watson Visual Recognition in Python</h1>

<h2>Introduction</h2>
<p>Here we will be training a Visual Recognition model that classify different kinds of dogs by running python code.</p>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size="3"><strong>Click on the links to go to the following sections:</strong></font>
<br>
<h2>Table of Contents</h2>
<ol>
    <li><a href="#ref1">IBM Watson Package</a></li>
    <li><a href="#ref2">Setting the API key for IBM Watson Visual Recognition</a></li>
    <li><a href="#ref3">Training the Classifier</a></li>
    <li><a href="#ref4">Testing the Classifier</a></li>
</ol>    
</div>

<a id="ref1"></a>
<h2>IBM Watson Package</h2>
In order to run this lab we need to import the following package.
<ul>
    <li>IBM Watson: which allows access to the Watson Visual Recognition API</li>
</ul>
The code below will install IBM Watson.

In [1]:
!pip install --upgrade ibm-watson

Requirement already up-to-date: ibm-watson in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (4.5.0)


<h3>Goal of this lab:</h3>

<p>In this lab, we will be creating a completely new image classifier using training images. We will train a custom classifier to identify between three different dog breeds (Golden Retriever, Beagle and Husky).</p>

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/dog-breed.png" width="480"/>

<a id="ref2"></a>
<h2>Setting the API key for IBM Watson Visual Recognition</h2>

<p>In order to use the IBM Watson Visual Recognition API, you will need the API key of the Visual Recognition instance that we had created in the Cloud instance.</p>

<p>Log into your IBM Cloud Account with the following link.</p> <a href="https://cocl.us/CV0101EN_IBM_Cloud_Login">https://cloud.ibm.com</a>
<ol>
    <li>Click on <b>Services</b></li>
    <li>Under Services, click on your Watson Visual Recognition Instance</li>
    <li>Copy the <b>API Key</b> and past it in the code cell below</li>
    <li>Then press "ctrl + enter" to run the code cell.</li>
</ol>

In [2]:
# Paste your API key for IBM Watson Visual Recognition below:
my_apikey = ''

<h4>Initialize Watson Visual Recognition</h4>
Lets create your own Watson Visual Recognition instance, it will allow you to make calls to the Watson Visual Recognition API.

In [3]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2018-03-19', 
                             authenticator=authenticator)


<p>We are going to train an Image Recognition model to classify different types of dog. The dataset that we are going to use are the zip files that we use below</p>

<ul>
    <li>beagle.zip</li>
    <li>husky.zip</li>
    <li>golden-retriever.zip</li>
</ul>

<a id="ref3"></a>
<h2>Training Classifier</h2>

<h4>Download the differerent breed of dog images as zip files</h4>
<p>We will use the <b>urlretrieve</b> method from the <b>urllib.request</b> library to download the dataset above.</p> 

In [4]:
import urllib.request

# Downloading Beagle dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Beagle.zip", 
                           "beagle.zip")

# Downloading Husky dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Husky.zip", 
                           "husky.zip")

# Downloading Golden Retriever dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever.zip", 
                           "goldenretriever.zip") #note that we should remove any hyphens from the zip file name

('goldenretriever.zip', <http.client.HTTPMessage at 0x7fe91b8951d0>)

<p>Lets train our Visual Recognition model to recognize the three breeds of dogs using the <b>create_classifier</b> method from the Watson Image Recognition API.</p>

In [5]:
import json
with open('beagle.zip', 'rb') as beagle, \
     open('goldenretriever.zip', 'rb') as gretriever, \
     open('husky.zip', 'rb') as husky:
        response = visrec.create_classifier(name="dogbreedclassifier",
                                        positive_examples={'beagle': beagle, \
                                                           'goldenretriever': gretriever, \
                                                           'husky': husky})
print(json.dumps(response.get_result(), indent=2))

{
  "classifier_id": "dogbreedclassifier_1490310135",
  "name": "dogbreedclassifier",
  "status": "training",
  "owner": "0347fd9c-758b-48c5-adc3-03eb9eec3b69",
  "created": "2020-07-01T16:44:22.807Z",
  "updated": "2020-07-01T16:44:22.807Z",
  "classes": [
    {
      "class": "husky"
    },
    {
      "class": "goldenretriever"
    },
    {
      "class": "beagle"
    }
  ],
  "rscnn_enabled": false,
  "core_ml_enabled": true
}


In [6]:
#lets grab the classifier id
classifier_id = response.get_result()["classifier_id"]
classifier_id

'dogbreedclassifier_1490310135'

Depending on the number of images, it may take <b>a Several Minutes </b> for Watson to build a custom classifier. Please wait tell you get <b> Good to Go<b> 

In [ ]:
Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

<h4>List all (custom) classifiers</h4>

<h4>If the status is still training, please rerun the above cell and wait until you see ready</h4> 

In [ ]:
visrec.list_classifiers(verbose=True).get_result()

<a id="ref4"></a>
<h2>Testing Classifier</h2>
<p>Let's test the classifier, the function <b>getdf_visrec</b> below uses the method <b>classify</b> from Watson Visual Recognition API to upload the image to the classifier and give us a result in JSON(JavaScript Object Notation) format. Then we use the method <b>json_normalize</b> from the "Pandas" library in Python to turn the result into a table because it is more human readable.</p>

In [ ]:
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0.6',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever1_stacked.jpg">
<p>Let's test our Visual Recognition model on this picture of Golden Retriever</p>

In [ ]:
getdf_visrec(url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever1_stacked.jpg',
            classifier_ids=classifier_id)

## Lets Test with cats so that we can take different outcomes to make an unique one out of all

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/cat-2083492_960_720.jpg">
<p>Lets test our Visual Recognition model on this picture of cat</p>

In [ ]:
getdf_visrec(url = 'http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/cat-2083492_960_720.jpg',
            classifier_ids=classifier_id)

<p>Our model will mis-classify the cat in the picture because our custom visual recognition model is only trained for recognizing different breeds of dogs.</p>

<h4>Delete all classifiers</h4>
<p>If you want to delete you classifiers, lets get the classifier id tht you want to delete. The method <b>list_classifiers</b> from Watson Visual Recognition API list all the classifier in your IBM Cloud account.</p>

In [ ]:
import json

classifiers = visrec.list_classifiers(verbose=True).get_result()['classifiers']
print(json.dumps(classifiers, indent=2))

Just paste your classifier id and it will be deleted

In [ ]:
mycid = '' #the classifier id you want to delete
visrec.delete_classifier(classifier_id = mycid)

<a id="ref5"></a>
<h2>Exercises</h2>
<p>For the following exercises you are going to train a Custom Visual Recognition Classifier to recognize fast food items, in particular it will be able to classify food items into <b>Burger</b>, <b>Fries</b> or <b>Coke</b>.<p>

<h3>Question 1: Optional</h3>
<p>After training your custom classifier, you might want to delete all the custom classifiers from your account, write a piece of code to use the *list_classifier* and *delete_classifier* method to delete all the custom classifiers in your account.</p>

<p><b>Note!</b> This question is optional, if there is a classifier that you have trained and do not want to delete, skip this question.</p>

In [ ]:
# Write your code below and press Shift+Enter to execute 


Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:

# The code below will delete all classifiers in your account
for i in visrec.list_classifiers().get_result()['classifiers']:
    print('Deleting ...' + i['classifier_id'])
    visrec.delete_classifier(classifier_id=i['classifier_id'])
-->

<h3>Question 2</h3>
<p>The link to the data set for Burger, Fries and Coke is given below</p> 
    
<p>We will use the <b>urlretrieve</b> method from the <b>urllib.request</b> library to download the dataset below.</p> 


<ul>
    <li>https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Burger.zip</li>
    <li>https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Fries.zip</li>
    <li>https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Pizza.zip</li>
</ul>

In [ ]:
# Write your code below and press Shift+Enter to execute 



Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
import urllib.request

#Downloading Burger dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Burger.zip", 
                           "burger.zip")

#Downloading Fries dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Fries.zip", 
                           "fries.zip")

#Downloading Pizza dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Pizza.zip", 
                           "pizza.zip")
-->

<h3>Question 3.1</h3>
<p>Now we have the dataset, use the <b>create_classifier</b> method to create your fast food classifier.</p>

In [ ]:
# Write your code below and press Shift+Enter to execute 


Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
my_apikey = '<paste your api key here>'

from ibm_watson import VisualRecognitionV3

visrec_fast_food = VisualRecognitionV3(version = '2019-01-01', 
                             iam_apikey = my_apikey)

import json
with open('burger.zip', 'rb') as burger, \
     open('fries.zip', 'rb') as fries, \
     open('pizza.zip', 'rb') as pizza:
    new_response = visrec_fast_food.create_classifier(name="fastfoodclassifier",
                                        positive_examples={'burger': burger, \
                                                           'fries': fries, \
                                                           'pizza': pizza})

    print(json.dumps(new_response.get_result(), indent=2))
-->

<h3>Question 3.2</h3>
<p>Since we will need the classifier_id, grab the classifier_id from the response of create_classifier and store it into a variable.</p>

In [ ]:
# Write your code below and press Shift+Enter to execute 
#lets grab the classifier id

Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
#lets grab the classifier id
fast_food_classifier_id = new_response.get_result()['classifier_id']
-->

<h3>Question 4</h3>
<p>Get a url of a picture of fast food and use the <b>getdf_visrec</b> function to classify the picture. <b>Before that, please make sure that your model is trained and ready </b>

In [ ]:
# Write your code below and press Shift+Enter to check if your model is ready to go 


Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
gStatus = visrec.get_classifier(classifier_id=fast_food_classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")
-->

In [ ]:
# Write your code below and press Shift+Enter to execute 


Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
getdf_visrec(url = 'fast_food_image_url',
            classifier_ids=fast_food_classifier_id)
-->

<h1>Thank you for completing this notebook</h1>
You can read more about Watson Visual Recognition APIs from the following link.
<a href="https://cloud.ibm.com/apidocs/visual-recognition?code=python">https://cloud.ibm.com/apidocs/visual-recognition</a>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Get IBM Watson Studio free of charge!</h2>
    <p><a href="https://cocl.us/NotebooksPython101bottom"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Logo/BottomAd.png" width="750" align="center"></a></p>
</div>

<h3>About the Authors:</h3>
<p>This notebook was written by <a href="https://www.linkedin.com/in/yi-leng-yao-84451275/" target="_blank" >Yi Yao</a> and revised by Nayef Abou Tayoun
<p><a href="https://www.linkedin.com/in/yi-leng-yao-84451275/" target="_blank">Yi Yao</a> is a Data Scientist and Software Engineer at IBM, and holds a Masters in Statistics. His research focused on Cloud Computing, Machine Learning and Computer Vision.</p>
<p>Nayef Abou Tayoun is a Cognitive Data Scientist at IBM, and pursuing a Master's degree in Artificial Intelligence.</p>
<hr>
<p>Copyright &copy; 2019 IBM Developer Skills Network. This notebook and its source code are released under the terms of the <a href="https://cognitiveclass.ai/mit-license/">MIT License</a>.</p>